## K-Nearest Neighbors

This is one of the simpliest and easiest to understand algorithms. It can be used for both classification and regression tasks, but is more common in classification, so we will focus there. The principles, though, can be used in both cases and sklearn supports both.

Basically, here is the algorithm:

1. Define $k$
2. Define a distance metric - usually Euclidean distance
3. For a new data point, find the $k$ nearest training points and combine their classes in some way - usually voting - to get a predicted class

That's it!

**Some of the benefits:**

* Doesn't really require any training in the traditional sense. You just need a fast way to find the nearest neighbors.
* Easy to understand 

**Some of the negatives:**

* Need to define k, which is a hyper-parameter, so it can be tuned with cross-validation. A higher value for k increases bias and a lower value increases variance.
* Have to choose a distance metric and could get very different results depending on the metric. Again, you can use cross-validation.
* Doesn't really offer insights into which features might be important.
* Can suffer with high dimensional data due to the curse of dimensionality.

**Basic assumption:**

* Data points that are close are similar for our target


### Curse of Dimensionality

Basically, the curse of dimensionality means that in high dimensions, it is likely that close points are not much closer than the average distance, which means being close doesnt mean much. In high dimensions, the data becomes very spread out, which creates this phenomenon. 

There are so many good resources for this online, that I won't go any deeper. Here is one you might look at:

http://blog.galvanize.com/how-to-manage-dimensionality/

### Euclidean Distance

For vectors, q and p that are being compared (these would be our feature vectors):

$$\sqrt{\sum_{i=1}^{N}{(p_{i} - q_{i})^2}}$$


### SKlearn Example



In [15]:
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report, accuracy_score, mean_squared_error

In [3]:
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.20, random_state=42)
clf = KNeighborsClassifier()
gridsearch = GridSearchCV(clf, {"n_neighbors": [1, 3, 5, 7, 9, 11], "weights": ['uniform', 'distance'], 
                                'p': [1, 2, 3]}, scoring='f1')
gridsearch.fit(X_train, y_train)
print("Best Params: {}".format(gridsearch.best_params_))
y_pred_train = gridsearch.predict(X_train)
print("Train F1: {}".format(f1_score(y_train, y_pred_train)))
print("Test Classification Report:")
y_pred_test = gridsearch.predict(X_test)
print(classification_report(y_test, y_pred_test))
print("Train Accuracy: {}\tTest accuracy: {}".format(accuracy_score(y_train, y_pred_train),
                                                     accuracy_score(y_test, y_pred_test)))

Best Params: {'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}
Train F1: 0.9606837606837607
Test Classification Report:
             precision    recall  f1-score   support

          0       0.97      0.88      0.93        43
          1       0.93      0.99      0.96        71

avg / total       0.95      0.95      0.95       114

Train Accuracy: 0.9494505494505494	Test accuracy: 0.9473684210526315


### Voting Methods

* Majority Voting: After you take the $k$ nearest neighbors, you take a "vote" of those neighbors' classes. The new data point is classified with whatever the majority class of the neighbors is. If you are doing a binary classification, it is recommended that you use an odd number of neighbors to avoid tied votes. However, in a multi-class problem, it is harder to avoid ties. A common solution to this is to decrease $k$ until the tie is broken.
* Distance Weighting: Instead of directly taking votes of the nearest neighbors, you weight each vote by the distance of that instance from the new data point. A common weighting method is $\hat{y} = \frac{\sum_{i=1}^Nw_iq_i}{\sum_{i=1}^N w_i}$, where $w_i=\frac{1}{\sum_{i=1}^{N}{(p_{i} - q_{i})^2}}$, or one over the distance between the new data point and the training point. The new data point is added into the class with the largest added weight. Not only does this decrease the chances of ties, it also reduces the effect of a skewed representation of data.

### Distance Metrics
Euclidean distance, or the 2-norm, is a very common distance metric to use for $k$-nearest neighbors. Any $p$-norm can be used (the $p$-norm is defined as $||\mathbf{x}_i-\mathbf{y}_i||_p = (\sum_{i=1}^N|x_i-y_i|^p)^\frac{1}{p}$. For categorical data, however, this can be a problem. For example, if we have encoded a feature for car color from red, blue, and green to 0, 1, 2, how can the "distance" between green and red be measured? You could make dummy variables, but if a feature has 15 possible categories, that means adding 14 more variables to your feature set, and we run into the curse of dimensionality. There are several ways to confront this issue, but they are beyond the scope of this lecture. But be aware of the effect categorical features can have on your nearest neighbors classifier.

### Search Algorithm

Imagine the data set contains 2000 points. A brute-force search for the 3 nearest neighbors to one point does not take very long. But if the data set contains 2000000 points, a brute-force search can become quite costly, especially if the dimension of the data is large. Other search algorithms sacrifice an exhaustive search for a faster run time. Structures such as KDTrees (see https://en.wikipedia.org/wiki/K-d_tree) or Ball trees (see https://en.wikipedia.org/wiki/Ball_tree) are used for faster run times. While we won't dive into the details of these structures, be aware of them and how they can optimize your run time (although, training time does increase).

### Radius Neighbors Classifier

This is the same idea as a $k$ nearest neighbors classifier, but instead of finding the $k$ nearest neighbors, you find all the neighbors within a given radius. Setting the radius requires some domain knowledge; if your points are closely packed together, you'd want to use a smaller radius to avoid having nearly every point vote.

### K Neighbors Regressor
To change our problem from classification to regressing, all we have to do is find the weighted average of the $k$ nearest neighbors. Instead of taking the majority class, we calculate a weighted average of these nearest values, using the same weighting method as above.

Let's try predicting the area of tissue based on the other features using a $k$-neighbors regressor.

In [19]:
target = data.data[:,3]
X = data.data[:,[0,1,2,4,5,6,7,8]]
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.20, random_state=42)
reg = KNeighborsRegressor()
gridsearch = GridSearchCV(reg, {"n_neighbors": [1, 3, 5, 7, 9, 11], "weights": ['uniform', 'distance'], 
                                'p': [1, 2, 3]}, scoring='neg_mean_squared_error')
gridsearch.fit(X_train, y_train)
print("Best Params: {}".format(gridsearch.best_params_))
y_pred_train = gridsearch.predict(X_train)
y_pred_test = gridsearch.predict(X_test)
print("Train MSE: {}\tTest MSE: {}".format(mean_squared_error(y_train, y_pred_train),
                                           mean_squared_error(y_test, y_pred_test)))

Best Params: {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
Train MSE: 0.0	Test MSE: 878.1482686614424


### Outlier Detection
In $k$-nearest neighbors, the data is naturally clustered. Within these clusters, we can find the average distance between points (either exhaustively or from the centroid of the cluster). If we find a few points that are much farther than the average distance to other points or to the centroid, it is reasonable (but not always correct) to think they could be outliers. We can use this process on new data points as well. 